In [61]:
def csv_FileReader(csvFile):
    import csv
    with open(csvFile, 'r') as file:
        reader = csv.reader(file)
        next(reader)
        accountsList = []
        for row in reader:
            accountsList.append(row)
        return accountsList

def csv_FileWriter(csvFile):
    import csv
    global customerList
    with open(csvFile, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["username","checking_id","checking_balance","savings_id","savings_balance","credit_id","credit_balance","credit_limit"])
        for cust in customerList:
            writer.writerow([cust.username, cust.checking.accountID, cust.checking.balance, cust.savings.accountID, cust.savings.balance, cust.credit.accountID, cust.credit.balance, cust.credit.credit_limit])
            #print(cust)
    
csvFile = 'accounts.csv'
csv_FileReader(csvFile)

[['amohr', '1337', '43.0', '0666', '101.45', '1729', '5000.0', '5000.0'],
 ['bbaggins', '2890', '15345.49', '2941', '4345.0', '3021', '0.0', '50000.0'],
 ['emusk',
  '0001',
  '21588738.0',
  '0002',
  '1000.0',
  '0003',
  '435678.6',
  '10000000.0']]

In [92]:
class Account:
    '''
    Represents a generic financial account

    attributes: accountID a string of 4 integers, balance as a float or int, interest as a float or int
    '''
    def __init__(self, accountID = '0000', balance = 0, interest = 0):

        if not isinstance(accountID, str):
            raise TypeError('Account id must be a string')
        
        if len(accountID) != 4:
            raise ValueError('Account id must contain 4 digits')

        for number in accountID:
            if number not in '0123456789':
                raise ValueError('Account id must contain only numbers')
        
        if not isinstance(balance, (int, float)):
            raise TypeError('balance must be an integer or float')
        
        if not isinstance(interest, (int, float)):
            raise TypeError('interest must be an integer or float')
        
        if interest < 0:
            raise ValueError('Interest cannot be negative')
        
        self.accountID = accountID
        self.balance = balance
        self.interest = interest
    
    def __str__(self):
        return f'Account ID: {self.accountID} Balance: {self.balance} Interest: {self.interest}%'

    def deposit(self, depositAmount):
        if not isinstance(self, Credit):
            self.balance += depositAmount
        

    def withdrawl(self, withdrawlAmount):
        if not isinstance(self, Credit):
            if withdrawlAmount <= self.balance:
                self.balance -= withdrawlAmount
            else:
                print('Withdrawing all funds.')
                self.balance = 0

class Checking(Account):
    '''
    Represents a checking account inheriting the Account class

    attributes: inherited attributes from Account class
    '''
    def __init__(self, accountID = '0000', balance = 0):
        super().__init__(accountID, balance)
        self.interest = 0

class Savings(Account):
    '''
    Represents a savings account inheriting the Account class

    attributes: inherited attributes from Account class
    '''
    def __init__(self, accountID = '0000', balance = 0):
        super().__init__(accountID, balance)
        self.interest = 1

class Credit(Account):
    '''
    Represents a credit account inheriting the Account class

    attributes: inherited attributes from Account class and a credit_limit which is an int or float
    '''
    def __init__(self, accountID = '0000', balance = 0, interest = 0, credit_limit = 0):
        super().__init__(accountID, balance, interest)
        
        if not isinstance(credit_limit, (int, float)):
            raise TypeError('Credit limit must be an int or float')
        
        if credit_limit < balance:
            raise ValueError("Credit limit must be greater than or equal to the balance")

        self.credit_limit = credit_limit
    
    def creditCharge(self, amount):
        if amount + self.balance > self.credit_limit:
            raise ArithmeticError("Cannot charge credit account. Amount exceeds credit limit.")
        self.balance += amount
        return
    
    def creditPayment(self, amount):
        if amount > self.balance:
            raise ArithmeticError("Amount to pay is greater than the balance in the credit account")
        self.balance -= amount

    def __str__(self):
        s = super().__str__()
        return f'{s} Credit limit: {self.credit_limit}'

class Customer:
    '''
    Respresents a customer account

    attributes: username as a string, checking as a Checking object, savings as a Savings object, credit as a Credit object
    '''        
    def __init__(self, username = '', checking = Checking() , savings = Savings(), credit = Credit()):
        
        if not isinstance(username, str):
            raise TypeError('username must be a string')
        
        if not isinstance(checking, Checking):
            raise TypeError('checking must be a Checking object')

        if not isinstance(savings, Savings):
            raise TypeError('savings must be a Savings object')

        if not isinstance(credit, Credit):
            raise TypeError('credit must be a Credit object')

        self.username = username
        self.checking = checking
        self.savings = savings
        self.credit = credit
    
    def __str__(self):
        return f'Customer: {self.username}, Checking Balance: {self.checking.balance}, Savings Balance: {self.savings.balance}, Credit Balance: {self.credit.balance}'

In [63]:
def importCSV():
    while True:
        csvFile = input("Enter the address of the csv file")
        try:
            accountInfo = csv_FileReader(csvFile)
            break
        except:
            print("Either the file you gave could not be found, or the CSV file could not be read. Try again. \n")
    global customerList
    customerList = []
    for row in accountInfo:
        createChecking = Checking(row[1], float(row[2]))
        createSavings = Savings(row[3], float(row[4]))
        createCredit = Credit(row[5], float(row[6]), 0, float(row[7]))
        createCustomer = Customer(row[0], createChecking, createSavings, createCredit)
        customerList.append(createCustomer)
    #for cust in customerList:
        #print(cust)
        
importCSV()


Either the file you gave could not be found, or the CSV file could not be read. Try again. 



KeyboardInterrupt: Interrupted by user

In [75]:
def viewCustomers():
    global customerList
    print("Customers:")
    for cust in customerList:
        print(cust)
    print("")

In [73]:
def CustomerSelection():
    
    global customerList

    while True:
        print(f"Enter customer username or type 'exit' to return to the menu:\n", flush = True)
        requestedUsername = input()

        if requestedUsername == 'exit' or requestedUsername == 'Exit': 
            return 'exit'

        for cust in customerList:
            if cust.username == requestedUsername:
                return cust

        print(f'No such username exists. Try again\n')

In [66]:
def deposit():
    
    customer = CustomerSelection()
    selectedAccount = 0
    if customer == 'exit': return

    print(f'Hi {customer.username}')
    while True:
        print("Press 1 to deposit into checking, Press 2 to deposit into savings, type 'exit' to cancel\n", flush = True)
        userInput = input("Type 1 or 2")
        if userInput == 'exit' or userInput == 'Exit': return
        if userInput == "1":
            selectedAccount = 1
            print(f'Selected checking:')
            print(f'Current balance: {customer.checking.balance}\n')
            break
        if userInput == "2": 
            selectedAccount = 2
            print(f'Selecting savings:')
            print(f'Current balance: {customer.savings.balance}\n')
            break

    while True:
        print("Please enter an amount to deposit to the account or type 'exit' to return to the menu", flush = True)
        userInput = input("Please enter an amount")
        if userInput == 'exit' or userInput == 'Exit': return
        try:
            amount = float(userInput)
            if selectedAccount == 1:
                customer.checking.deposit(amount)
                print(f"Depositing ${amount} in checking account")
                print(f"New checking balance: {customer.checking.balance}\n")
            if selectedAccount == 2:
                customer.savings.deposit(amount)
                print(f"Depositing ${amount} in savings account")
                print(f"New savings balance: {customer.savings.balance}\n")
            return
        except ValueError:
            print("Inputed amount is not a valid number\n")
        except:
            print("Error in depositing in account. Try again\n")

In [79]:
def withdrawl():
    
    customer = CustomerSelection()
    selectedAccount = 0
    if customer == 'exit': return

    print(f'Hi {customer.username}')
    while True:
        print("Press 1 to withdrawl from checking, Press 2 to withdrawl from savings, type 'exit' to cancel\n", flush = True)
        userInput = input("Enter 1 for checking or 2 for savings")
        if userInput == 'exit' or userInput == 'Exit': return
        if userInput == '1':
            selectedAccount = 1
            print(f'Selected checking:')
            print(f'Current balance: {customer.checking.balance}\n')
            break
        if userInput == '2': 
            selectedAccount = 2
            print(f'Selecting savings:')
            print(f'Current balance: {customer.savings.balance}\n')
            break

    while True:
        print("Please enter an amount to withdrawl from the account or type 'exit' to return to the menu", flush = True)
        userInput = input("Please enter an amount")
        if userInput == 'exit' or userInput == 'Exit': return
        try:
            amount = float(userInput)
            if selectedAccount == 1:
                customer.checking.withdrawl(amount)
                print(f"Withdrawing ${amount} from checking account")
                print(f"New Checking Balance: {customer.checking.balance}\n")
            if selectedAccount == 2:
                customer.savings.withdrawl(amount)
                print(f"Withdrawing ${amount} from savings account")
                print(f"New Savings Balance: {customer.savings.balance}\n")
            return
        except ValueError:
            print("Inputed amount is not a valid number\n")
        except:
            print("Error in depositing in account. Try again\n")

In [96]:
def creditChargeInterface():
    
    customer = CustomerSelection()
    
    if customer == 'exit': return

    print(f'Hi {customer.username}')
    print(f'Current balance: {customer.credit.balance}')
    print(f'Credit Limit: {customer.credit.credit_limit}\n')
    while True:
        print("Please enter an amount to charge to the credit account or type 'exit' to return to the menu\n", flush = True)
        userInput = input("Please enter an amount")
        if userInput == 'exit' or userInput == 'Exit': return
        try:
            amount = float(userInput)
            customer.credit.creditCharge(amount)
            print(f"Charging ${amount} to credit account")
            print(f"New credit balance: {customer.credit.balance}\n")
            return
        except ValueError:
            print("Inputted amount is not a valid number\n")
        except:
            print("Error in charging credit account. Try again\n")



In [87]:
def creditPaymentInterface():
    
    customer = CustomerSelection()
    if customer == 'exit': return

    print(f'Hi {customer.username}')
    print(f'Current balance: {customer.credit.balance}')
    print(f'Credit Limit: {customer.credit.credit_limit}\n')
    selectedAccount = 0
    while True:
        print("Type 1 to pay from checking account. Type 2 to pay from savings account. Or type 'exit' to return to the menu\n", flush = True)
        userInput = input("Please type 1 or 2")
        if userInput == 'exit' or userInput == 'Exit': return
        if userInput == "1":
            print("Checking Account Selected")
            print(f"Current Checking Balance: {customer.checking.balance}")
            selectedAccount = 1
            break
        if userInput == "2":
            print("Savings Account Selected")
            print(f"Current Savings Balance: {customer.savings.balance}")
            selectedAccount = 2
            break

    while True:
        print("Please enter an amount to pay to the credit account or type 'exit' to return to the menu\n", flush = True)
        userInput = input("Please enter an amount")
        if userInput == 'exit' or userInput == 'Exit': return
        try:
            amount = float(userInput)
            if selectedAccount == 1:
                if amount > customer.checking.balance:
                    print("Amount entered is more than amount in checking account. Try again")
                    print(f"Checking Balance: {customer.checking.balance}\n")
                    continue
                customer.checking.withdrawl(amount)
                customer.credit.creditPayment(amount)
                print(f"Transferring ${amount} from checking account to credit account")
                print(f"New Checking balance: {customer.checking.balance}")
                print(f"New Credit balance: {customer.credit.balance}\n")
                
            if selectedAccount == 2:
                if amount > customer.savings.balance:
                    print("Amount entered is more than amount in savings account. Try again")
                    print(f"Savings Balance: {customer.savings.balance}\n")
                    continue
                customer.savings.withdrawl(amount)
                customer.credit.creditPayment(amount)
                print(f"Transferring ${amount} from savings account to credit account")
                print(f"New Savings balance: {customer.savings.balance}")
                print(f"New Credit balance: {customer.credit.balance}\n")
            return
        except ValueError:
            print("Inputted amount is not a valid number\n")
        except:
            print("Error in paying credit account. Try again\n")


In [70]:
def customerLookup():
    global customerList
    while True:
        print(f'Enter customer username:', flush=True) 
        requestedUsername = input()

        for cust in customerList:
            if cust.username == requestedUsername:
                return cust

        print(f'No such username. Try again')

csv_FileWriter(csvFile)

In [71]:
def exitInterface():
    while True:
        print("Please enter a file address to save data to", flush = True)
        userInput = input("Please enter an address")
        try:
            csv_FileWriter(userInput)
            print("Accounts saved. Exiting")
            return
        except:
            print("Error in writing data to file try again \n")

In [97]:
def interface():
    importCSV()
    print("Accounts loaded successfully \n")
    choice = ''
    while choice != '6':
        print('Select an option:', flush=True)
        print("1: Deposit ", flush=True)
        print("2: Withdrawl", flush=True)
        print("3: Credit Card Charge", flush=True)
        print("4: Credit Card Payment", flush=True)
        print("5: View Customers", flush=True)
        print("6: Exit/Shut Down", flush=True)
        print(flush=True)

        choice = input("Select a menu option")

        if choice == '1':
            print("Selected Deposit")
            deposit()
            # Try: Deposit()
            # Except: print("An error occured")
        elif choice == '2':
            print("Selected Withdrawl")
            withdrawl()
        elif choice == '3':
            print("Selected Credit Card Charge")
            creditChargeInterface()
        elif choice == '4':
            print("Selected Credit Card Payment")
            creditPaymentInterface()
        elif choice == '5':
            viewCustomers()
        elif choice == '6':
            exitInterface()
        else:
            print("Select and actual option.")

interface()

Accounts loaded successfully 

Select an option:
1: Deposit 
2: Withdrawl
3: Credit Card Charge
4: Credit Card Payment
5: View Customers
6: Exit/Shut Down

Selected Deposit
Enter customer username or type 'exit' to return to the menu:

Hi amohr
Press 1 to deposit into checking, Press 2 to deposit into savings, type 'exit' to cancel

Press 1 to deposit into checking, Press 2 to deposit into savings, type 'exit' to cancel

Selected checking:
Current balance: 43.0

Please enter an amount to deposit to the account or type 'exit' to return to the menu
Depositing $5000.0 in checking account
New checking balance: 5043.0

Select an option:
1: Deposit 
2: Withdrawl
3: Credit Card Charge
4: Credit Card Payment
5: View Customers
6: Exit/Shut Down

Selected Deposit
Enter customer username or type 'exit' to return to the menu:

Hi amohr
Press 1 to deposit into checking, Press 2 to deposit into savings, type 'exit' to cancel

Selecting savings:
Current balance: 101.45

Please enter an amount to depo